# Handling units in Python with `pint`

Keeping track of appropriate units is vitally important in any physical simulation.  In molecular dynamics programs, units can be especially tricky, since the masses, length scales and energy scales are so far removed from our daily experience.

To help us keep track of units we can use a package called `pint`

___
## Installing and importing `pint`

In [ ]:
# uncomment this to install pint
#pip install pint

In [ ]:
from pint import UnitRegistry
unit = UnitRegistry()

The `unit` object now holds all of the units we need to do our calculations.  

## Making quantity objects

Instead of storing our positions, masses, velocities, forces, etc. as just numbers, we can store them as **quantities** by attaching units to them like this:

In [ ]:
velocity = 1000*unit.m/unit.sec
time_step = 0.001*unit.ps
position = 0*unit.angstrom

print("Velocity is:",velocity)
print("Time step is:", time_step)
print("Position is:", position)

And now we can manipulate these quantities safely, without worrying about a unit mismatch:

In [ ]:
new_position = position + time_step*velocity
print(new_position)

## Handling quantity objects

Once you have a quantity there are a couple of things you might want to do:

### Display them in a different unit

In [ ]:
print("New position in meters:",new_position.to(unit.meters))

note that you can only select a compatible unit.  You can't change a distance into a volume, for instance:

In [ ]:
# this code will throw an error
a = 1.5*unit.meters
a.to(unit.liters)

In [ ]:
# but this code won't
a = 1.5*unit.meters**3
a.to(unit.liters)

### Strip the units entirely

To get the value contained in a quantity, you just need to specify which units you would like:

In [ ]:
my_mass = 1.1*unit.kg
my_mass_val = my_mass.to(unit.g).magnitude

print("my_mass_val = ",my_mass_val)
print("type of my_mass_val:",type(my_mass_val))

## Quantities with `numpy` arrays

These operations work well with numpy arrays too:

In [ ]:
import numpy as np

velocity = np.random.random((20,3))*unit.m/unit.sec
position = np.zeros((20,3))*unit.angstrom
new_position = position + time_step*velocity

print(new_position)

Note that even though this object is a `pint.quantity` object, it still has access to all of the `numpy` array functions:

In [ ]:
type(new_position)

print("Mean:", new_position.mean())
print("Argmin: ", new_position.argmin())

And it even keeps track of the units when calling `numpy` operations!

In [ ]:
np.square(new_position)

As an example, let's try to do something a little more complicated:  apply a force to update a particle's velocity.  You should recall from class that:

\begin{equation} \overrightarrow{v}(t+\Delta t) \approx \overrightarrow{v}(t) + \frac{\overrightarrow{F_{net}}(t)}{m} \Delta t \end{equation}

A common unit to describe energies in molecular simulations is **kcal/mol**. Since forces are derivatives of the energy with respect to positions, we will use units of **kcal/(mol Å)**.  In pint, this is `unit.kcal/unit.mol/unit.angstrom`.  Below, we will use `unit.kcal/unit.mol/unit.angstrom * unit.molecule` to cancel out the [substance] unit introduced by `unit.mol`. 

For the masses, we will use **Daltons** (`unit.daltons`).

As above, for space we will use `unit.angstrom` and for time we will use `unit.ps`, making our velocity units: `unit.angstrom/unit.ps`.

In [ ]:
v_old = 0*unit.angstrom/unit.ps
m = 10 * unit.daltons
dt = 0.002 * unit.ps
f_net = 100 * (unit.kcal/unit.mol)/unit.angstrom*unit.molecule  # the unit.molecule is needed to cancel out the [substance] unit introduced by unit.mol 

v_new = v_old + f_net/m*dt
print(v_new)